<a href="https://colab.research.google.com/github/srmty09/Deep-Learning/blob/main/Fine_Tuning/unsloth_finetuning_blog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning Using Unsloth.

1. Model Loading.

In [ ]:
!pip install unsloth

In [ ]:
from unsloth import FastLanguageModel
# fastlanguage model is where all the language model present.
import torch
max_seq_len = 512
load_in_4bit = True # for using 4bit quantized model.

Some of 4 bit quantized models:
1. "unsloth/Meta-Llama-3.1-8B-bnb-4bit"
2. "unsloth/Mistral-Small-Instruct-2409"
3. "unsloth/phi-4"
4. "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"

In [ ]:
model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/phi-4", # which model we want to use
    max_seq_length = max_seq_len,
    load_in_4bit = load_in_4bit
    )

Model names ending with *unsloth-bnb-4bit* indicate they are unsloth dynamic 4bit models, these models are slightly more vram than standard BitsBytes 4bit models but offer better accuracy.

# Dataset
___

- we need to create a dataset usually with 2 columns Question and answer.
- we can create synthetically generated data and structure the dataset using chatgpt.
- we need to know the purpose of dataset, to know for what cause we need the dataset and how we will use it.

## Chat Template with Unsloth


In [ ]:
from unsloth.chat_templates import CHAT_TEMPLATES
print(list(CHAT_TEMPLATES.keys()))

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [ ]:
def formatting_prompts_func(examples):
  convos = examples["messages"]
  texts = [tokenizer.apply_chat_template(convo,tokenize=False,add_generation_prompt=False) for convo in convos]
  return {"text":texts,}

In [ ]:
examples = {
    "messages": [
        [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What is Bayes theorem?"},
            {"role": "assistant", "content": "Bayes theorem describes conditional probability."}
        ],
        [
            {"role": "system", "content": "You are a math tutor."},
            {"role": "user", "content": "Explain eigenvalues."},
            {"role": "assistant", "content": "Eigenvalues measure scaling of linear transforms."}
        ]
    ]
}


In [ ]:
formatting_prompts_func(examples)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("HuggingFaceH4/ultrachat_200k",split = "train_sft")

# dataset.map is used to map the dataset to the required format to train the model.
dataset = dataset.map(formatting_prompts_func,batched = True)

In [ ]:
print(dataset)

In [ ]:
dataset[1]["text"]

# some datasets
1. Alpaca Dataset
2. HuggingFaceH4/ultrachat_200k
3.

In [ ]:
dataset = load_dataset("vicgalle/alpaca-gpt4",split = "train")
print(dataset)

In [ ]:
dataset[0]

In [ ]:
dataset = dataset.map(formatting_prompts_func,batched = True)

# Finetuning Llama-3!!


In [ ]:
!pip install "unsloth[colab-new] @ git+http://github.com/unslothai/unsloth.git"
!pip uninstall -y xformers
!pip install trl==0.8.6 --no-deps

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [ ]:
model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

In [ ]:
# peft: parameter efficient fine tuning

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj",
                      "gate_proj","up_proj","down_proj"],
    lora_alpha = 16,
    lora_dropout=0,
    bias = "none",
    use_gradient_checkpointing= "unsloth",
    random_state = 3407,
    use_rslora=False,
    loftq_config=None
)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("vicgalle/alpaca-gpt4",split = "train")
print(dataset)

In [ ]:
# from unsloth import to_sharegpt
# dataset = to_sharegpt(
#     dataset,
#     merged_prompt = "{instruction}[[\nYour input is:\n{input}]]",
#     output_column_name="output",
#     conversation_extension=3,
# )

In [ ]:
print(dataset)

In [ ]:
dataset[0]

In [ ]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset.select(range(10000)),
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        remove_unused_columns = False,
    ),
)


In [ ]:
trainer_stats = trainer.train()